In [15]:
%load_ext autoreload
%autoreload 2

In [1]:
# pathmap.'/scratch/enis/data/nna/database/allFields_dataV5.pkl'

In [17]:
import pandas as pd
from datetime import datetime

import numpy as np

from pathlib import Path
import matplotlib.pylab as pl

from nna import visutils 


channels_audio = CHANNELS_AUDIO = 2 
# CONFIGS
class pathMap():

    def __init__(self) -> None:
        scratch = '/scratch/enis/data/nna/'
        home = '/home/enis/projects/nna/'
        self.data_folder = scratch + 'database/'
        self.aggreated_data_cache_path = scratch + 'results/vis_export/'

        self.exp_dir = '/home/enis/projects/experiments/megantaxo/runs/augment'

        self.clipping_results_path = Path(scratch +
                                          'clipping_info/all-merged_2021-12-24/')

        self.output_dir = scratch + 'real/'

        self.file_properties_df_path = self.data_folder + 'allFields_dataV7.pkl'

        self.results_folder = home + 'results/'
        self.vis_output_path = self.results_folder + 'vis/yfitloiq-V1/'

def setup_configs():
    pathmap = pathMap()

    config = {}
    versiontag = 'yfitloiq-V1'
    config['versiontag']  =versiontag

    id2name={'1-0-0': 'biophony',
     '1-1-0': 'bird',
     '1-1-10': 'songbirds',
     '1-1-7': 'duck-goose-swan',
     '0-0-0': 'anthrophony',
     '1-3-0': 'insect',
     '1-1-8': 'grouse-ptarmigan',
     '0-2-0': 'aircraft',
     '3-0-0': 'silence'}
    
    
    generic_id2name = list(id2name.items())
    id2name = {}
    for k, v in generic_id2name:
        id2name[f'{versiontag}-{k}'] = v

    config['id2name'] = id2name
    config['input_data_freq'] = '10S'
    # FREQS to reduce results
    # config['output_data_freq'] = '270min'
    config['output_data_freq'] = 'D'

    classname2colorindex = {

        'anthrophony':14,
        'auto':15,
        'aircraft':9,

        'wind':18,
        'running water':0,
        'silence':1,

        'bird':4,
        'songbirds':5,
        'duck-goose-swan':16,
        'grouse-ptarmigan':17,

        'biophony':8,
        'insect':10,
        'mammal':2,

    }

    config['classname2colorindex'] = classname2colorindex
    config['prob2binary'] = True

    # cmap to use
    aCmap = pl.cm.tab20
    norm_cmaps = visutils.add_normal_dist_alpha(aCmap)
    # cached results
    # src='/home/enis/projects/nna/src/scripts/'
    # csv_file=src+'dalton/03/aggregated/multi9-V1_prob2binary=False_output-data-freq=270min_prob.csv'

    config['norm_cmaps'] = norm_cmaps
    
    return pathmap, config



def setup(args, pathmap, region_location_years=None):

    file_properties_df = pd.read_pickle(pathmap.file_properties_df_path)

    #important to keep them in order
    file_properties_df.sort_values(by=['timestamp'], inplace=True)

    # delete older than 2016

    # delete older than 2016
    fromtime = datetime(2016, 1, 1, 0)
    file_properties_df = file_properties_df[
        file_properties_df.timestamp >= fromtime]
    

#     if not region_location:
#         # region_location = [('anwr','49'),('prudhoe','11'),('prudhoe','26')]
#         region_location = tuple(
#             sorted(
#                 set(
#                     zip(file_properties_df.region.values,
#                         file_properties_df.locationId.values))))

    if region_location_years is None:

        region_location_years = tuple(
            sorted(
                set(
                    zip(file_properties_df.region.values,
                        file_properties_df.locationId.values,
                        file_properties_df.year.values,))))

        region_location_years = [i for i in region_location_years if 'stinchcomb' not in i[0]]

    return region_location_years,file_properties_df

In [18]:
# file_properties_df['timestamp']

In [19]:
def sigmoid(data):
    return 1 / (1 + np.exp(-data))

### FUNC from export_cli.py
def get_cached_file_name(config, df_type='prob'):

    csv_file_name_parts = (config['versiontag'],
                           f'prob2binary={config["prob2binary"]}',
                           f'output-data-freq={config["output_data_freq"]}',
                           df_type)
    csv_file_name = '_'.join(csv_file_name_parts)

    csv_file_name = csv_file_name + '.csv'

    return csv_file_name
        





In [20]:
# cached_pred = '/scratch/enis/data/nna/results/vis_export/yfitloiq-V1/anwr/31/2021/yfitloiq-V1_prob2binary=True_output-data-freq=D_prob.csv'
# data = pd.read_csv(cached_pred)
# data['TimeStamp'] = pd.to_datetime(  # type: ignore
#     data['TimeStamp'], format='%Y-%m-%d_%H:%M:%S')
# data = data.set_index('TimeStamp')

In [21]:
# data.dropna()

In [22]:

def vis_preds_with_clipping_local(region_location,year, config, file_properties_df,
                            pathmap,norm_cmaps,cached_preds,save_fig=True,channel=2):


    no_result = {}

    for region, location_id in region_location:
        # print(region, all_regions.index(region),'location_id',location_id)
        filtered_files = file_properties_df[file_properties_df.region == region]
        filtered_files = filtered_files[filtered_files.locationId ==
                                        location_id]
        filtered_files = filtered_files[filtered_files.durationSec > 0]
        # filtered_files = filtered_files[filtered_files.year == year]
        # print(filtered_files)
        # print(filtered_files['timestamp'])
        figures_axes, no_result_paths = visutils.vis_preds_with_clipping(
#         test = visutils.vis_preds_with_clipping(
            region,
            location_id,
            filtered_files,
            config['input_data_freq'],
            config['output_data_freq'],
            config['id2name'].keys(),
            norm_cmaps,
            pathmap.output_dir,
            pathmap.clipping_results_path,
            pathmap.vis_output_path,
            config['id2name'],
            clipping_threshold=1.0,
            prob2binary_flag=True,
            pre_process_func=sigmoid,
            classname2colorindex=config.get('classname2colorindex',None),
            cached_pred=cached_preds[(region,location_id)],
#             cached_pred='',
            cached_pred_version='V2',
            save_fig=False,
            channel=channel)
#         return test
    
        no_result[region,location_id] = no_result_paths

        fig,ax=figures_axes[0]
        classname2colorindex=config['classname2colorindex']
        visutils.add_legend(ax,classname2colorindex,norm_cmaps,legend_ax_index=0,)
        fig.tight_layout()
        fig.subplots_adjust(top=0.95)


        if save_fig:
#             year='2019'
            fig_dir = Path(pathmap.vis_output_path) / ("Freq-" + config['output_data_freq']) / config['versiontag'] / region
            fig_dir.mkdir(parents=True, exist_ok=True)
            fig_path = fig_dir / ("_".join([location_id, str(year)]) + "." + "png")

            fig.savefig(fig_path)
            print(f'figured saved to {fig_path}')
        del fig


    return no_result



In [23]:
# result=np.zeros((4,3))
# config['output_data_freq']

In [24]:
# cached_pred = '/scratch/enis/data/nna/results/vis_export/182tahb6-V1/dempster/12/aggregated/2019/182tahb6-V1_prob2binary=True_output-data-freq=270min_prob.csv'
# df_freq = visutils.load_cached_preds(cached_pred,
#                                     region,
#                                     '12',
#                                     True,
#                                     config['output_data_freq'],
#                                     version='V1')


In [25]:
# file_properties_df

In [26]:
# no_result_paths[0][2][9]
print('test')

test


In [27]:
# dd11[dd11.year=='2021'].sort_values(by='timestamp')

In [28]:
# region_location_years

In [35]:
%matplotlib agg
%matplotlib agg

pathmap, config = setup_configs()

region_location_years, file_properties_df = setup('', pathmap, None)
region_location_years = [('anwr', '31', '2021'), ('prudhoe', '11', '2021'),('prudhoe', '23', '2021')]
for region, location, year in region_location_years:
    # if region in ['prudhoe','anwr']:
    #     continue
# #testing prudhoe/12
    # try:
    if region != '' and location != '':
        region_location = [(region, location)]
    else:
        print('Region and location are not given, we will do all of them.')
        region_location = None

    csv_file_name = get_cached_file_name(config, df_type='prob')

    cached_preds = {}
    for region, location in region_location:
        cache_path = (Path(pathmap.aggreated_data_cache_path) / config['versiontag'] / region /
                      location / 'aggregated' / str(year))
        file_abs_path = cache_path / csv_file_name
        if not file_abs_path.exists():
            cache_path = (Path(pathmap.aggreated_data_cache_path) / config['versiontag'] / region /
                          location / str(year))
            file_abs_path = cache_path / csv_file_name
            if not file_abs_path.exists():
                print("ERROR missing file",file_abs_path)
        cached_preds[(region, location)] = (file_abs_path)
        print((cache_path / csv_file_name))

    print('now')

    norm_cmaps = config['norm_cmaps']
    no_result_paths = vis_preds_with_clipping_local(
        region_location, year,config, file_properties_df, pathmap, norm_cmaps,cached_preds,channel=channels_audio)
# except:
    #     print(':(')


/scratch/enis/data/nna/results/vis_export/yfitloiq-V1/anwr/31/2021/yfitloiq-V1_prob2binary=True_output-data-freq=D_prob.csv
now
monthi 0
month.index DatetimeIndex(['2019-07-04', '2019-07-05', '2019-07-06', '2019-07-07',
               '2019-07-08', '2019-07-09', '2019-07-10', '2019-07-11',
               '2019-07-12', '2019-07-13', '2019-07-14', '2019-07-15',
               '2019-07-16', '2019-07-17', '2019-07-18', '2019-07-19',
               '2019-07-20', '2019-07-21', '2019-07-22', '2019-07-23',
               '2019-07-24', '2019-07-25', '2019-07-26', '2019-07-27',
               '2019-07-28', '2019-07-29', '2019-07-30', '2019-07-31'],
              dtype='datetime64[ns]', freq=None)
monthi 1
month.index DatetimeIndex(['2019-07-01', '2019-07-02', '2019-07-03', '2019-07-04',
               '2019-07-05', '2019-07-06', '2019-07-07', '2019-07-08',
               '2019-07-09', '2019-07-10', '2019-07-11', '2019-07-12',
               '2019-07-13', '2019-07-14', '2019-07-15', '2019-07-16',

ValueError: view limit minimum -0.5717944050000001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f969a3ee1e0> (for post_execute):


ValueError: view limit minimum -0.5717944050000001 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

In [ ]:
# Path('/scratch/enis/data/nna/results/vis_export/3rk9ayjc-V1/anwr/31/2019/3rk9ayjc-V1_prob2binary=True_output-data-freq=270min_prob.csv').parts

In [ ]:
print('test')

In [ ]:
!/scratch/enis/conda/envs/speechEnv/bin/python /home/enis/projects/nna/src/nna/slack_message.py -t "cpu job ended" -m 'vis export ended'

In [ ]:
# fig

In [ ]:
fig,ax=figures_axes[0]


csv_path = 'dalton03labels.csv'

(letters,y_multi_labels_by_month,
 x_multi_label_by_month, 
 classname2colorindex,color_indexes)=visutils.load_enis_labels4bars(csv_path,config['classname2colorindex'])

norm_cmaps=visutils.add_equally_spaced_bars(letters,y_multi_labels_by_month,
                        x_multi_label_by_month,ax,color_indexes,norm_cmaps)

visutils.add_legend(ax,classname2colorindex,norm_cmaps,legend_ax_index=0,)
fig.tight_layout()
fig.subplots_adjust(top=0.95)




In [15]:
file_properties_df = pd.read_pickle('/scratch/enis/data/nna/database/allFields_dataV5.pkl')


In [22]:
ff=file_properties_df[file_properties_df['region']=='anwr']
ff=ff[ff['site_id']=='31']

In [23]:
ff.sort_values(by=['timestamp'], inplace=True)

In [24]:
ff

,site_id,locationId,site_name,recorderId,hour_min_sec,year,month,day,region,timestamp,durationSec,timestampEnd
/tank/data/nna/real/anwr/31/2019/S4A10297_20190504_000000.flac,31,31,,S4A10297,000000,2019,05,04,anwr,2019-05-04 00:00:00,4560,2019-05-04 01:16:00
/tank/data/nna/real/anwr/31/2019/S4A10297_20190504_011602.flac,31,31,,S4A10297,011602,2019,05,04,anwr,2019-05-04 01:16:02,4438,2019-05-04 02:30:00
/tank/data/nna/real/anwr/31/2019/S4A10297_20190504_043000.flac,31,31,,S4A10297,043000,2019,05,04,anwr,2019-05-04 04:30:00,4560,2019-05-04 05:46:00
/tank/data/nna/real/anwr/31/2019/S4A10297_20190504_054602.flac,31,31,,S4A10297,054602,2019,05,04,anwr,2019-05-04 05:46:02,4438,2019-05-04 07:00:00
/tank/data/nna/real/anwr/31/2019/S4A10297_20190504_093000.flac,31,31,,S4A10297,093000,2019,05,04,anwr,2019-05-04 09:30:00,4560,2019-05-04 10:46:00
...,...,...,...,...,...,...,...,...,...,...,...,...
/tank/data/nna/real/anwr/31/2019/S4A10297_20190903_054602.flac,31,31,,S4A10297,054602,2019,09,03,anwr,2019-09-03 05:46:02,4438,2019-09-03 07:00:00
/tank/data/nna/real/anwr/31/2019/S4A10297_20190903_093000.flac,31,31,,S4A10297,093000,2019,09,03,anwr,2019-09-03 09:30:00,4560,2019-09-03 10:46:00
/tank/data/nna/real/anwr/31/2019/S4A10297_20190903_104602.flac,31,31,,S4A10297,104602,2019,09,03,anwr,2019-09-03 10:46:02,4438,2019-09-03 12:00:00
/tank/data/nna/real/anwr/31/2019/S4A10297_20190903_140000.flac,31,31,,S4A10297,140000,2019,09,03,anwr,2019-09-03 14:00:00,4560,2019-09-03 15:16:00
